In [1]:

import  numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import xgboost as xgb 
from sklearn import metrics as m
from thundersvm import SVC as svmgpu
import calculateWeightUsingGa2 as aresult
import pandas as pd
import itertools
import random
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder

def swapcolumns(trainval, testval, coldindexval):
    trainval[trainval != coldindexval] = 5
    testval[testval != coldindexval] = 5

    trainval[trainval == coldindexval] = 0
    trainval[trainval == 5] = 1

    testval[testval == coldindexval] = 0
    testval[testval == 5] = 1

    return trainval, testval


randomseed=42
np.random.seed(randomseed)

In [2]:

xtest=np.array(pd.read_csv('../dataset/xtest.txt'))
xtrain=np.array(pd.read_csv('../dataset/xtrain.txt'))
ytest_original=np.array(pd.read_csv('../dataset/ytest.txt')).ravel()
ytrain_original =np.array(pd.read_csv('../dataset/ytrain.txt')).ravel()

# data=datasets.load_wine()
# x=data.data
# y=data.target

# data = pd.read_csv("../dataset/seeds_dataset.txt", sep="\t", header=None)
# data = shuffle(data)
# le = LabelEncoder()
# data.iloc[:, -1] = le.fit_transform(data.iloc[:, -1])
# x = data.iloc[:, :-1]
# y = data.iloc[:, -1]
# print(np.unique(y))

# data = pd.read_csv("../dataset/ionosphere.data",  header=None)
# data = shuffle(data)
# le = LabelEncoder()
# data.iloc[:, -1] = le.fit_transform(data.iloc[:, -1])
# x = data.iloc[:, :-1]
# y = data.iloc[:, -1]
# print(np.unique(y))




# xtrain,xtest,ytrain_original,ytest_original=train_test_split(x,y,random_state=randomseed,test_size=0.3) 

ytrain=ytrain_original.copy()
ytest=ytest_original.copy() 

# ytrain, ytest = swapcolumns(ytrain, ytest, 2)

In [3]:


clf=[]
acc=[]
finalacc=[]
ypredproba_all=[]
ypredconfprob_all=[]


#================================================= 

# Class 2
# ===========================
ytrain = ytrain_original.copy()
ytest = ytest_original.copy()
ytrain,ytest= swapcolumns(ytrain,ytest,2)
#=================================================

rf=RandomForestClassifier(random_state=randomseed, n_estimators=10)
rf.fit(xtrain,ytrain)
print('original score',m.f1_score(ytest,rf.predict(xtest),average='weighted'))


xgbc=xgb.XGBClassifier(random_state=randomseed,n_estimators=100)
xgbc.fit(xtrain,ytrain)
xgbpred=xgbc.predict(xtest)
print('original score 2',m.f1_score(ytest,xgbpred,average='weighted'))


#================================================= 

rf=RandomForestClassifier(random_state=randomseed, n_estimators=10)
rf.fit(xtrain,ytrain)
rfpred=rf.predict(xtest)
print(m.f1_score(ytest,rfpred,average='weighted'))

clf.append(rf)
acc.append(m.f1_score(ytest,rfpred,average='weighted'))
ypredproba_all.append(rf.predict_proba(xtest)[:,0])

confmat = m.confusion_matrix(ytest, rfpred)
confsumh = np.sum(confmat, axis=0)
propconfmat = confmat.copy()
for i in range(propconfmat.shape[0]):
    propconfmat[:, i] = 100 * propconfmat[:, i] / confsumh[i]
ypredconfprob_all.append(propconfmat.ravel() / 100)


#=================================================
svc=svmgpu(random_state=randomseed,probability=True,C=100,gamma=0.0001)
svc.fit(xtrain,ytrain)

svcpred=svc.predict(xtest)
print(m.f1_score(ytest,svcpred,average='weighted'))

clf.append(svc)
acc.append(m.f1_score(ytest,svcpred,average='weighted'))
ypredproba_all.append(svc.predict_proba(xtest)[:,0])

confmat = m.confusion_matrix(ytest, rfpred)
confsumh = np.sum(confmat, axis=0)
propconfmat = confmat.copy()
for i in range(propconfmat.shape[0]):
    propconfmat[:, i] = 100 * propconfmat[:, i] / confsumh[i]
ypredconfprob_all.append(propconfmat.ravel() / 100)

#=================================================
xgbc=xgb.XGBClassifier(random_state=randomseed,n_estimators=100)
xgbc.fit(xtrain,ytrain)

xgbpred=xgbc.predict(xtest)
print(m.f1_score(ytest,xgbpred,average='weighted'))


clf.append(xgbc)
acc.append(m.f1_score(ytest,xgbpred,average='weighted'))
ypredproba_all.append(xgbc.predict_proba(xtest)[:,0])

confmat = m.confusion_matrix(ytest, rfpred)
confsumh = np.sum(confmat, axis=0)
propconfmat = confmat.copy()
for i in range(propconfmat.shape[0]):
    propconfmat[:, i] = 100 * propconfmat[:, i] / confsumh[i]
ypredconfprob_all.append(propconfmat.ravel() / 100)

#=================================================
#=================================================
# generate combinations of features 12,6
comb = list(itertools.combinations(np.arange(0, 12, 1), 7))

# generate 50 random numbers
randnums = []
for i in range(50):
    randnums.append(random.randrange(0, len(comb)))

print(randnums)

comb = np.array(comb)[randnums, :]


for i in range(len(comb)):
    print(i, " ==================== ", comb[i])

    rf = RandomForestClassifier(random_state=randomseed, n_estimators=50)
    rf.fit(xtrain[:, comb[i]], ytrain)
    rfpred = rf.predict(xtest[:, comb[i]])
    print(m.f1_score(ytest, rfpred,average='weighted'))

    clf.append(rf)
    acc.append(m.f1_score(ytest, rfpred,average='weighted'))
    ypredproba_all.append(rf.predict_proba(xtest[:, comb[i]])[:,0])

    confmat = m.confusion_matrix(ytest, rfpred)
    confsumh = np.sum(confmat, axis=0)
    propconfmat = confmat.copy()
    for i in range(propconfmat.shape[0]):
        propconfmat[:, i] = 100 * propconfmat[:, i] / confsumh[i]
    ypredconfprob_all.append(propconfmat.ravel() / 100)

    xgbmodel = xgb.XGBClassifier(random_state=randomseed, n_estimators=50)
    xgbmodel.fit(xtrain, ytrain)
    xgbmodelpred = xgbmodel.predict(xtest)
    print(m.f1_score(ytest, xgbmodelpred,average='weighted'))

    clf.append(xgbmodel)
    acc.append(m.f1_score(ytest, xgbmodelpred,average='weighted'))
    ypredproba_all.append(xgbmodel.predict_proba(xtest)[:,0])

    confmat = m.confusion_matrix(ytest, xgbmodelpred)
    confsumh = np.sum(confmat, axis=0)
    propconfmat = confmat.copy()
    for i in range(propconfmat.shape[0]):
        propconfmat[:, i] = 100 * propconfmat[:, i] / confsumh[i]
    ypredconfprob_all.append(propconfmat.ravel() / 100)

# #=================================================


original score 0.8886190707463169
original score 2 0.9114979197447021
0.8886190707463169
0.9178477001592273
0.9114979197447021
[731, 383, 219, 647, 33, 170, 222, 223, 64, 54, 248, 674, 141, 594, 363, 727, 121, 400, 686, 101, 452, 193, 464, 182, 380, 786, 533, 778, 361, 389, 543, 646, 164, 312, 152, 640, 65, 445, 613, 1, 714, 242, 555, 468, 300, 34, 773, 263, 277, 694]
0  ====================  [ 2  4  5  6  7  9 10]
0.6008058447394335
0.911721530549929
1  ====================  [ 0  3  4  5  6  8 10]
0.8589065255731922
0.911721530549929
2  ====================  [ 0  1  4  6  7  9 10]
0.8968253968253967
0.911721530549929
3  ====================  [ 1  4  5  6  7  9 10]
0.8518518518518519
0.911721530549929
4  ====================  [ 0  1  2  3  5  9 10]
0.8867589442471787
0.911721530549929
5  ====================  [ 0  1  3  5  6 10 11]
0.8867589442471787
0.911721530549929
6  ====================  [ 0  1  4  6  8  9 10]
0.8858436510798574
0.911721530549929
7  ====================  [ 0  1  4

In [4]:
# pd_clf=pd.DataFrame(list(clf))
pd_acc=pd.DataFrame(acc) 
pd_ypredproba_all=pd.DataFrame(np.array(ypredproba_all))
pd_ypredconfprob_all=pd.DataFrame(ypredconfprob_all)

In [5]:
classname='class2.txt'
pd_acc.to_csv('pd_acc_'+classname,sep=',',index=False) 
pd_ypredproba_all.to_csv('pd_ypredproba_all_'+classname,sep=',',index=False)
pd_ypredconfprob_all.to_csv('pd_ypredconfprob_all_'+classname,sep=',',index=False)

In [6]:
pd.DataFrame(ytrain_original).to_csv('ytrain_original.txt',sep=',',index=False)
pd.DataFrame(ytest).to_csv('ytest_class2.txt',sep=',',index=False)

In [7]:
# svc=svmgpu(random_state=randomseed,probability=True,C=100,gamma=0.0001)
# svc.fit(xtrain,ytrain)

# svcpred=svc.predict(xtest)
# print(m.f1_score(ytest,svcpred,average='weighted'))

# temppp=svc.predict_proba(xtest)

In [8]:
# m.f1_score(ytest, np.argmax(temppp, axis=1),average='weighted')